In [1]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

d:\Python_Scripts\series_temporais_prophet\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [124]:
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df['ds'] = pd.to_datetime(df['ds'])
df


,ds,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677
3,2007-12-13,8.072467
4,2007-12-14,7.893572
...,...,...
2900,2016-01-16,7.817223
2901,2016-01-17,9.273878
2902,2016-01-18,10.333775
2903,2016-01-19,9.125871


In [125]:
#cria a função para criar as estacoes do ano
def setSeasons(data):
    if data.month in [1, 2, 3]:
        return 1 # verão
    elif data.month in [4, 5, 6]:
        return 2 # outono
    elif data.month in [7, 8, 9]:
        return 3 # inverno
    else:
        return 4 # primavera

In [126]:
#cria nova feature regressor no df original
df['season'] = df['ds'].apply(setSeasons)

In [128]:
# cria do modelo
modelo = Prophet()

# adiciona regressor no modelo
modelo.add_regressor('season')

#treina o modelo
modelo.fit(df)

future = modelo.make_future_dataframe(periods=12, freq='M')

future.tail()

15:14:44 - cmdstanpy - INFO - Chain [1] start processing
15:14:45 - cmdstanpy - INFO - Chain [1] done processing


,ds
2912,2016-08-31
2913,2016-09-30
2914,2016-10-31
2915,2016-11-30
2916,2016-12-31


In [129]:
#cria nova feature regressor no df future
future['season'] = future['ds'].apply(setSeasons)
future.tail()

,ds,season
2912,2016-08-31,3
2913,2016-09-30,3
2914,2016-10-31,4
2915,2016-11-30,4
2916,2016-12-31,4


In [130]:
# predicao
forecast = modelo.predict(future)

forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,extra_regressors_additive,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
2912,2016-08-31,7.323546,6.836416,8.049680,7.135094,7.511485,0.105720,0.105720,0.105720,-0.013384,...,-0.066437,-0.066437,-0.066437,0.185541,0.185541,0.185541,0.0,0.0,0.0,7.429266
2913,2016-09-30,7.292424,6.979262,8.366816,7.068336,7.524109,0.417878,0.417878,0.417878,-0.013384,...,-0.069610,-0.069610,-0.069610,0.500873,0.500873,0.500873,0.0,0.0,0.0,7.710303
2914,2016-10-31,7.260265,7.331632,8.702125,6.993580,7.538588,0.756055,0.756055,0.756055,-0.040422,...,0.352370,0.352370,0.352370,0.444108,0.444108,0.444108,0.0,0.0,0.0,8.016321
2915,2016-11-30,7.229144,7.045749,8.453064,6.911791,7.551098,0.501288,0.501288,0.501288,-0.040422,...,-0.066437,-0.066437,-0.066437,0.608147,0.608147,0.608147,0.0,0.0,0.0,7.730432
2916,2016-12-31,7.196985,6.710565,8.153600,6.840020,7.564086,0.230270,0.230270,0.230270,-0.040422,...,-0.311857,-0.311857,-0.311857,0.582549,0.582549,0.582549,0.0,0.0,0.0,7.427255


In [106]:
forecast = forecast[forecast['ds'] > '2018-01-01']
df = df[df['ds'] > '2018-01-01']

In [134]:
from plotly import graph_objects as go
from plotly.subplots import make_subplots

# Cria um subplot com duas figuras, uma para o gráfico de tendência e outro para sazonalidades
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.0)

#Adiciona o gráfico de valores reais
fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='markers', name='Valores Reais', marker=dict(color='rgba(120,120,255, 0.6)')))

#Adiciona o gráfico de tendência
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Tendência Prevista'))

#Adiciona os gráficos de sazonalidade
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yearly'], mode='lines', name='Sazonalidade'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['season'], mode='lines', name='Regressor Estação Ano'))

#Personaliza o layout do gráfico
fig.update_layout(title="Previsão de valor da Bolsa com componentes sazonas e valores reais",
                  yaxis_title='Preço', showlegend=True, height=800)

fig.show()


In [ ]:
modelo.plot_components(forecast)